In [1]:
###############################################################################
### pthon imports                                                           ###
###############################################################################

from os import path
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import re

In [2]:
###############################################################################
### local imports                                                           ###
###############################################################################

import zachquire as zaq

In [3]:
def find_file(
    file_name='path/file.txt',
    cache=False,
    cache_age=None
    ):

    if cache==False:
        return None
    
    return path.exists(file_name)

In [35]:
def get_soup(
    url='https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    file_name='soupfiles/gitsearch.txt',
    cache=False,
    cache_age=None,
    soup_slurper='*'
):
    # if we already have the data, read it locally
    file_found = find_file(file_name=file_name, cache=cache, cache_age=cache_age)
    if file_found:
        with open(file_name) as f:
            return BeautifulSoup(f.read())

    # otherwise go fetch the data
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text)
    print(soup)
    slurps = soup.select(soup_slurper)
    print(len(slurps))
    print(str(slurps[0]))

    # save it for next time
    with open(file_name, 'w') as f:
        f.write(str(slurps[0]))
        if len(slurps)>1:
            for slurp in slurps[1:]:
                print(str(slurp))
                f.write('\n' + str(slurp))
        
    with open(file_name) as f:    
        soup = BeautifulSoup(f.read())
#     pd.to_csv(slurps, header=None, index=False)

    return soup

In [37]:
soup = get_soup(
    url='https://github.com/search?o=desc&p=2&q=advent+of+code&s=stars&type=Repositories',
    cache=False,
    soup_slurper='.repo-list-item a',
    file_name='soupfiles/gitsearch.txt'
)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-9b5314213e37056ed87b0418056c4f2c.css" integrity="sha512-7mtunHqp/Bw0ND9akjJME8XCh0WPm3HAXOSeX7skL0qGAhpdfzkQvYcujYcwNPTpWKeKMFUGZGtvnEkcczFgwQ==" media="all" rel="stylesheet"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-4e9f27fa33341743f730ae7b0e33eff5.css" integrity="sha512-CmoegizWCUR1jC94Y2eukVQIFxJ9Gx

In [24]:
# print(soup)

In [38]:
re_url = re.compile(r'"url"\:"(.+?)"')
urllist = re_url.findall(str(soup))

urllist

['https://github.com/mfikes/advent-of-code',
 'https://github.com/dlew/aoc-2017',
 'https://github.com/mstksg/advent-of-code-2017',
 'https://github.com/kodsnack/advent_of_code_2017',
 'https://github.com/Diggsey/aoc2018',
 'https://github.com/mstksg/advent-of-code-2019',
 'https://github.com/mhanberg/advent-of-code-elixir-starter',
 'https://github.com/borkdude/advent-of-cljc',
 'https://github.com/BartoszMilewski/AofC2017',
 'https://github.com/zero-to-mastery/coding_challenge-24']

In [ ]:
# souprepo = get_soup(
#     url = 'https://github.com/mfikes/advent-of-code',
#     file_name = 'soupfiles/mfikes_advent-of-code.html',
#     soup_slurper='html',
#     cache = False
# )


In [25]:
# souprepo = get_soup(
#     url = 'https://github.com/Diggsey/aoc2018',
#     file_name = 'soupfiles/diggsey_aoc2018.html',
#     soup_slurper='html',
#     cache = False
# )

In [26]:
# souprepo = get_soup(
#     url = 'https://github.com/mhanberg/advent-of-code-elixir-starter',
#     file_name = 'soupfiles/mhanberg_advent-of-code-elixir-starter.html',
#     soup_slurper='html',
#     cache = False
# )

In [39]:
# Library imports for sending request, parsing HTML, and pandas
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
# Example list of URLs from GitHub
urls = [
    "https://github.com/mfikes/advent-of-code",
    "https://github.com/ryanorsinger/intro-to-machine-learning-workshop",
    'https://github.com/Diggsey/aoc2018',
    "https://github.com/kjeliasen/AdventOfCode",
    'https://github.com/mhanberg/advent-of-code-elixir-starter'
]
output = []
for url in urls:
    request = get(url)
    soup = BeautifulSoup(request.text)
    title = url.split("/")[-1]
    language_info = soup.select_one(".repository-lang-stats-numbers")
    languages = [language.get_text() for language in language_info.select(".lang")]
    percentages = [percentage.get_text() for percentage in language_info.select(".percent")]
    readme = soup.select_one("#readme article")
    repo_data = {}
    repo_data["title"] = title
    repo_data["readme"] = readme
    repo_data["languages"] = languages
    repo_data["language_percentages"] = percentages
    output.append(repo_data)
df = pd.DataFrame(output)
df.to_csv("soupfiles/all_repo_data.csv")

In [40]:
df

,language_percentages,languages,readme,title
0,[100.0%],[Clojure],"<article class=""markdown-body entry-content p-...",advent-of-code
1,[100.0%],[Jupyter Notebook],"<article class=""markdown-body entry-content p-...",intro-to-machine-learning-workshop
2,[100.0%],[Rust],None,aoc2018
3,"[96.0%, 4.0%]","[Jupyter Notebook, Python]","<article class=""markdown-body entry-content p-...",AdventOfCode
4,[100.0%],[Elixir],"<article class=""markdown-body entry-content p-...",advent-of-code-elixir-starter


In [41]:
re_repo = re.compile(r'https\://github.com/(.+?)$')
repolist = [re_repo.findall(url)[0] for url in urllist]
repolist

['mfikes/advent-of-code',
 'dlew/aoc-2017',
 'mstksg/advent-of-code-2017',
 'kodsnack/advent_of_code_2017',
 'Diggsey/aoc2018',
 'mstksg/advent-of-code-2019',
 'mhanberg/advent-of-code-elixir-starter',
 'borkdude/advent-of-cljc',
 'BartoszMilewski/AofC2017',
 'zero-to-mastery/coding_challenge-24']

In [42]:
filepath = "datafiles/data.json"
zaq.scrape_github_data(repos=repolist, filepath=filepath)
git_json = pd.read_json(filepath)

mfikes/advent-of-code
dlew/aoc-2017
mstksg/advent-of-code-2017
kodsnack/advent_of_code_2017
Diggsey/aoc2018


MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?

In [23]:
git_json

,language,readme_contents,repo
0,Python,# Awesome Advent of Code\n\nThis is a collecti...,Bogdanp/awesome-advent-of-code
1,Rust,BurntSushi's 2018 Advent of Code solutions\n==...,BurntSushi/advent-of-code
2,Coq,Advent of Code 2018 in Coq\n==================...,Lysxia/advent-of-coq-2018
3,Python,# Advent of Code 2018\n\nMy solutions to the A...,fogleman/AdventOfCode2018
4,Haskell,Advent of Code 2018\n===================\n\n*[...,mstksg/advent-of-code-2018
5,C++,# advent2018-fast\n\n[Advent of Code 2018](htt...,Voltara/advent2018-fast
6,Python,Advent of Code data\n===================\n\n|p...,wimglenn/advent-of-code-data
7,Python,Advent of Code\n==============\n\nThis is a co...,ChrisPenner/Advent-Of-Code-Polyglot
8,Python,# Advent of Other People's Code\nFetches rando...,jaksi/advent-of-other-peoples-code
9,Python,using Neural Networks (SSD) on Tensorflow.\n\n...,molyswu/hand_detection
